# Análise exploratória dos dados

### Imports

In [82]:
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [2]:
df = pd.read_csv('../../data/base_livros.csv')

In [3]:
print(f'Tamanho da base de dados: {df.shape[0]}')
df.head()

Tamanho da base de dados: 1000


,title,price,rating,availability,category,image
0,It's Only the Himalayas,45.17,2,In stock,Travel,https://books.toscrape.com/media/cache/27/a5/2...
1,Full Moon over Noahâs Ark: An Odyssey to Mou...,49.43,4,In stock,Travel,https://books.toscrape.com/media/cache/57/77/5...
2,See America: A Celebration of Our National Par...,48.87,3,In stock,Travel,https://books.toscrape.com/media/cache/9a/7e/9...
3,Vagabonding: An Uncommon Guide to the Art of L...,36.94,2,In stock,Travel,https://books.toscrape.com/media/cache/d5/bf/d...
4,Under the Tuscan Sun,37.33,3,In stock,Travel,https://books.toscrape.com/media/cache/98/c2/9...


### Há livros duplicados?

In [4]:
dup = df[['title']].value_counts().reset_index(name = 'qtd')
dup = dup[dup['qtd'] > 1]

df.merge(dup, on = 'title', how = 'inner')

,title,price,rating,availability,category,image,qtd
0,The Star-Touched Queen,46.02,5,In stock,Fantasy,https://books.toscrape.com/media/cache/53/5e/5...,2
1,The Star-Touched Queen,32.30,5,In stock,Fantasy,https://books.toscrape.com/media/cache/d3/0d/d...,2


### Estatísticas gerais da coleção

In [161]:
print(f"Total de livros: {df['title'].nunique()}")
print(f"Categorias distintas: {df['category'].nunique()}")
print(f"Preço médio: {round(df['price'].mean(), 2)}")

dist_rating = df[['rating']].value_counts().reset_index(name = 'qtd').sort_values('rating', ascending = False)
dist_rating['rating'] = dist_rating['rating'].astype(str)

fig = px.bar(dist_rating, x = 'qtd', y = 'rating', text = 'qtd')
fig.update_layout(plot_bgcolor = 'white', height = 350, width = 800, title = 'Distribuição de rating', barcornerradius = 4)
fig.update_traces(width = 0.6, textposition = 'outside', textfont = dict(size = 12), marker_color = 'steelblue')
fig.update_xaxes(title = '', tickfont = dict(size = 10, color = '#d4d4d4'), range = (0, 300))
fig.update_yaxes(title = '')
fig.add_hline(y = -0.5, line_width = 1, line_color = '#d4d4d4')
fig.show()

Total de livros: 999
Categorias distintas: 50
Preço médio: 35.07


### Faixas de preço

In [29]:
preco_min = df['price'].min().round(2)
preco_max = df['price'].max().round(2)

print(f"Preço mínimo: {(preco_min)}")
print(f"Preço máximo: {(preco_max)}")

Preço mínimo: 10.0
Preço máximo: 59.99


In [143]:
fig = px.histogram(df, x = 'price', nbins = 100)
fig.update_layout(plot_bgcolor = 'white', title = 'Distribuição de preços')
fig.update_xaxes(range = (0, preco_max + 10), title = 'Preço')
fig.update_yaxes(title = '', tickfont = dict(color = '#d4d4d4'))
fig.update_traces(marker_color = 'steelblue')
fig.add_hline(y = 0, line_width = 1)
fig.show()

### Categorias

In [95]:
df_categorias = df.groupby('category').agg({'title':'count', 'price':'mean', 'rating': 'mean'}).round(2).reset_index()
df_categorias.columns = ['category', 'n_books', 'mean_price', 'mean_rating']

max_books = df_categorias['n_books'].max()
price_range = preco_max - preco_min

df_categorias['decis_qtd'] = pd.qcut(df_categorias['n_books'], 10, duplicates = 'drop')
decis = sorted(df_categorias['decis_qtd'].unique())
dict_decis = {}
for i in range(9):
    dict_decis[decis[i]] = (i + 1) / 10

df_categorias['score_qtd'] = df_categorias['decis_qtd'].map(dict_decis).astype(float)
df_categorias['score_price'] =  1 - ((df_categorias['mean_price'] - preco_min) / price_range)
df_categorias['score_rating'] = (df_categorias['mean_rating'] - 1) / 4

df_categorias['category_score'] = df_categorias['score_qtd'] * df_categorias['score_price'] * df_categorias['score_rating']
df_categorias.sort_values('category_score', ascending = False).round(2).head()
df_categorias['qtd_livros'] = df_categorias['n_books'].apply(lambda x: f"{x} {'livro' if x == 1 else 'livros'}")
# df_categorias

In [160]:
# title = """
# O score da categoria (último gráfico) foi calculado ponderando as seguintes regras: Maior quantidade de livros, menor média de preço, maior média de classificação<br>
# <sup>Apenas categorias com mais de 1 livro</sup>"""

title = dict(
    text="O score da categoria (último gráfico) foi calculado ponderando as seguintes regras: Maior quantidade de livros, menor média de preço, maior média de classificação",
    subtitle=dict(text="Apenas categorias com mais de 1 livro"),
    x=0.05,
    xanchor="left"
)

fig = make_subplots(rows = 1, cols = 4,
                    subplot_titles = ('Maior quantidade de livros', 'Menores médias de preço', 'Maiores médias de ratings', 'Melhores scores'))

dict_features = {'n_books':[1, False, 200], 'mean_price':[2, True, 60], 'mean_rating':[3, False, 10], 'category_score':[4, False, 0.5]}

for k, v in dict_features.items():

    tmp = df_categorias[df_categorias['n_books'] > 1][['category', k, 'qtd_livros']].sort_values(k, ascending = v[1]).head().sort_values(k, ascending = not v[1])
    if k in['mean_price', 'mean_rating']:
        tmp['text'] = tmp[k].astype(str) + ' (' + tmp['qtd_livros'] + ')'
    else:
        tmp['text'] = tmp[k].round(3)

    fig.add_trace(go.Bar(x = tmp[k],
                         y = tmp['category'],
                         text = tmp['text'],
                         orientation = 'h'),
                  row = 1, col = v[0])
    fig.update_xaxes(range = (0, v[2]), tickfont = dict(color = '#d4d4d4'), row = 1, col = v[0])
    fig.add_hline(y = -0.5, line_width = 1, line_color = '#d4d4d4', row = 1, col = v[0])

fig.update_layout(plot_bgcolor = 'white', showlegend = False, title = title, barcornerradius = 4, margin = dict(t = 150))
fig.update_traces(width = 0.5, textposition = 'outside', marker_color = 'steelblue')
fig.show()